In [3]:
# encoding: utf-8
import fileinput
import json
import copy
import nltk
import pprint
from nltk.stem import WordNetLemmatizer
import re
import string
from collections import Counter
import urllib
from nltk.corpus import sentiwordnet as swn
from contractions import CONTRACTION_MAP
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
 


In [4]:
def read_tweets(filenames):
    # https://stackoverflow.com/questions/24754861/unicode-file-with-python-and-fileinput
    # fileinput.input(filename, openhook=fileinput.hook_encoded("utf-8")).
    # raw = url.read().decode('windows-1252')
    with fileinput.input((filenames)) as f:
        for line in f:
            tweet = json.loads(line)
            tweets.append(tweet)
        return tweets
    


In [5]:
tweets = []
filenames = ['data/MelbourneTweets0.txt', 
             'data/MelbourneTweets2.txt']

tweets = read_tweets(filenames)
tweets_experimental = copy.deepcopy(tweets)

In [6]:
# Source code from https://github.com/dipanjanS/text-analytics-with-python
def expand_contractions(sentence, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence

In [7]:
for t in tweets:
    # https://stackoverflow.com/questions/7395789/replacing-a-weird-single-quote-with-blank-string-in-python
    t['text_expanded'] = expand_contractions(t['text'], CONTRACTION_MAP)
    t['text_removed'] = re.sub(r'http\S+', '', t['text_expanded'])
    t['text_removed'] = re.sub(r'\n', '', t['text_removed'])
    t['text_removed'] = re.sub(r'RT', '', t['text_removed'])

In [8]:
# # this isn't working due to encoding issue
# # more on this later
# print(expand_contractions("next time someone asks why i'm moving i'll reply", CONTRACTION_MAP))
# print(tweets[3]['text'])
# expand_contractions(tweets[3]['text'], CONTRACTION_MAP)


In [9]:
# handles @mention, make lowercase
tknzr = nltk.tokenize.casual.TweetTokenizer(preserve_case=False, 
                                            strip_handles=True, 
                                            reduce_len=True)
for t in tweets:
    t['text_tokenized'] = tknzr.tokenize(t['text_removed'])
# tweets_tokenized = [tknzr.tokenize(tweet) for tweet in tweets_minus_escape]

In [10]:
# def extract_geo():
#     for tweet in tweets:
#         if(tweet['geo'] is not None):
#             t = {
#                 "coordinates": tweet['coordinates']['coordinates'],
#                 "date": tweet['created_at'],
#                 "text": tweet['text']
#             }
#             geo_tweets.append(t)

# def geo_minmax(coord):
#     x = [g['coordinates'][coord] for g in geo_tweets]
#     xmin, xmax = min(x), max(x)
#     print(xmin)
#     print(xmax)


In [11]:
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma

In [12]:
lemmatizer = WordNetLemmatizer()
# tweets_minus_stop = []
stopword_list = nltk.corpus.stopwords.words('english')

for t in tweets:
    not_a_stopword = []
    for word in t['text_tokenized']:
        word = lemmatize(word)
        if word not in stopword_list:
            not_a_stopword.append(word)
    t['text_tokens'] = not_a_stopword


In [13]:
# >> c = Counter()
# >>> for xs in seq:
# ...     for x in set(xs):
# ...         c[x] += 1
# ...
# >>> c
# Counter({'a': 2, 'c': 2, 'b': 2, 'd': 1})

token_counter = Counter()
for t in tweets:
    for x in set(t['text_tokens']):
        token_counter[x] += 1

token_counter.most_common()

# print(token_counter.most_common())

[(':', 15458),
 ('…', 12353),
 ('.', 11106),
 ('?', 8995),
 (',', 7153),
 ('!', 4561),
 ('’', 4377),
 ('year', 2170),
 ('look', 2047),
 ('still', 1842),
 ('guy', 1668),
 ('...', 1658),
 ('hand', 1600),
 ('get', 1589),
 ('-', 1585),
 ('war', 1570),
 ('hold', 1479),
 ('60', 1446),
 ('donald', 1406),
 ('comfortable', 1403),
 ('melania', 1394),
 ('one', 1282),
 ('like', 1218),
 ('(', 1111),
 ('go', 1067),
 ('#', 1066),
 (')', 1018),
 ('think', 1008),
 ('people', 998),
 ('make', 996),
 ('new', 878),
 ('time', 855),
 ('see', 844),
 ('&', 835),
 ('good', 806),
 ('day', 784),
 ('"', 774),
 ('#singtoprachaya', 759),
 ('#kristperawat', 757),
 ('#singtokrist', 756),
 ('love', 723),
 ('watch', 692),
 ('say', 684),
 ('work', 678),
 ('thank', 663),
 ('great', 640),
 ('u', 629),
 ('need', 628),
 ('know', 622),
 ('“', 619),
 ('would', 590),
 ('come', 579),
 ('want', 570),
 ("'", 566),
 ('take', 564),
 ('show', 560),
 ('via', 542),
 ('today', 513),
 ('follow', 512),
 ('”', 504),
 ('first', 503),
 ('bac

In [14]:
# remove punctuations
# have to get encoding right to resolve this issue
# tweets_final = []
# for tweet in tweets_minus_stop:
#     tweet = [''.join(c for c in s if c not in string.punctuation) for s in tweet]
#     tweet = [t for t in tweet if t]
#     tweets_final.append(tweet)

In [15]:
def read_words(filename):
    with open(filename) as f:
        lines = f.readlines()
        return lines

negative_words = read_words('./data/negative.txt')
positive_words = read_words('./data/positive.txt')

In [24]:
count = 0
for index, t in enumerate(tweets):
    pos = 0
    neg = 0
    for word in t['text_tokens']:
        if word in negative_words:
            neg += 1
        elif word in positive_words:
            pos += 1
        else:
            pass
    t['text_score'] = pos - neg
    if t['text_score'] != 0:
        print(t['text_score'])
        count += 1

print(count)

0


In [19]:
for t in tweets:
    x.append(t['text_score'])

print(x)
# num_bins = 1
# n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
# plt.show()

[21, 22, 23, 4, 5, 6, 77, 8, 9, 10, 31, 32, 33, 34, 35, 36, 37, 18, 49, 50, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

dict